In [3]:
import numpy as np
import pandas as pd

In [4]:
emnist_train = pd.read_csv("/Users/jaladisaran/Downloads/emnist-letters-train.csv")
emnist_test = pd.read_csv("/Users/jaladisaran/Downloads/emnist-letters-test.csv")

In [73]:
train = emnist_train.to_numpy()
test = emnist_test.to_numpy()

In [139]:
X_train = train[:,1:]
X_test = test[:,1:]

In [140]:
Y_train = train[:,0]
Y_test = test[:,0]

In [141]:
X_train=X_train/255
X_test=X_test/255

In [142]:
Z_train= np.zeros((X_train.shape[0],len(np.unique(Y_train))))
Z_train.shape

(88799, 26)

In [143]:
for i in range(X_train.shape[0]):
    Z_train[i][Y_train[i]-1] = 1

Z_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [144]:
def function(X,W,b):
    z = np.dot(X,W)+b
    a = 1/(1+np.exp(-z))
    return a

def forward_prop(X,W1,b1,W2,b2):
    H1 = function(X,W1,b1)
    A  = function(H1,W2,b2)
    return A,H1

def err(A,Y):
    err = A-Y
    return err

def dsig(A):
    z = np.multiply(A,1-A)
    return z

def loss_function(X,Y,W1,b1,W2,b2):
    m=X.shape[0]
    a = forward_prop(X,W1,b1,W2,b2)[0]
    f1 = np.log(a)
    f2 = np.log(1-a)
    loss = -np.sum(np.multiply(Y,f1)+np.multiply((1-Y),f2))
    loss = loss/m
    return loss

def gradient_compute(X,Y,W1,b1,W2,b2):
    A,H1 = forward_prop(X,W1,b1,W2,b2)
    
    m = X.shape[0]
    dj_dw2 = np.dot(H1.T,np.multiply(err(A,Y),dsig(A)))/m
    dj_db2 = np.sum(np.multiply(err(A,Y),dsig(A)),axis =0)/m
    dj_dw1 = np.dot(X.T,np.multiply(np.dot(dj_db2,W2.T),dsig(H1)))/m
    dj_db1 = np.sum(np.multiply(np.dot(dj_db2,W2.T),dsig(H1)),axis=0)/m
    
    return dj_dw2,dj_db2,dj_dw1,dj_db1

def gradient_descent(X,Y,W1,b1,W2,b2,L,n):
    
    for i in range(n):
        loss = loss_function(X,Y,W1,b1,W2,b2)
        print(i,loss)
        
        dj_dw2 = gradient_compute(X,Y,W1,b1,W2,b2)[0]
        dj_db2 = gradient_compute(X,Y,W1,b1,W2,b2)[1]
        dj_dw1 = gradient_compute(X,Y,W1,b1,W2,b2)[2]
        dj_db1 = gradient_compute(X,Y,W1,b1,W2,b2)[3]
        
        W2 = W2 - L*dj_dw2
        W1 = W1 - L*dj_dw1
        b2 = b2 - L*dj_db2
        b1 = b1 - L*dj_db1
        
        
    return W1,b1,W2,b2

In [305]:
n=52
W1 = np.random.rand(X_train.shape[1],n)-0.5
b1 = np.random.rand(n)-0.5
W2 = np.random.rand(n,Z_train.shape[1])-0.5
b2 = np.random.rand(Z_train.shape[1])-0.5

In [306]:
W1_final,b1_final,W2_final,b2_final = gradient_descent(X_train,Z_train,W1,b1,W2,b2,0.5,100)

0 20.113518129830737
1 12.128256530462753
2 8.447870837100474
3 6.830105301477419
4 6.0753927136247325
5 5.648602001732244
6 5.373541813684724
7 5.182257356439907
8 5.042275374385433
9 4.935978664935393
10 4.852956337673967
11 4.7866573851200584
12 4.732754647791603
13 4.688276776025336
14 4.6511174287220065
15 4.619743180007364
16 4.5930121677628515
17 4.570057411399356
18 4.5502094137317695
19 4.532943447397474
20 4.517842814173151
21 4.504572709619562
22 4.492861291445612
23 4.482485741399072
24 4.4732618522481244
25 4.465036144495141
26 4.457679825804327
27 4.45108411108134
28 4.44515655983706
29 4.439818182867803
30 4.4350011368967
31 4.430646872978811
32 4.426704638286962
33 4.423130255430162
34 4.419885121455426
35 4.416935382024046
36 4.414251246235676
37 4.4118064151111795
38 4.409577602483876
39 4.40754413145407
40 4.405687592967112
41 4.403991555728029
42 4.402441318744272
43 4.40102369942877
44 4.399726851496726
45 4.398540107928623
46 4.397453845105399
47 4.396459364894191

In [307]:
NN_output = forward_prop(X_test,W1_final,b1_final,W2_final,b2_final)[0]
NN_output.shape

(14799, 26)

In [308]:
max = NN_output.max(axis=1)
for i in range(X_test.shape[0]):
    for j in range(len(np.unique(Y_train))):
        if NN_output[i][j] == max[i]:
            NN_output[i][j] = 1
        else:
            NN_output[i][j] = 0
            
NN_output

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [309]:
NN_predicted = np.empty_like(Y_test)
for i in range(X_test.shape[0]):
    for j in range(len(np.unique(Y_train))):
        if NN_output[i][j]==1:
            NN_predicted[i]=j+1
            
NN_predicted


array([ 2, 23, 22, ..., 15, 23, 25])

In [310]:
def accuracy(yout,ytest):
    m = yout.shape[0]
    count=0
    for i in range(m):
        if yout[i]==ytest[i]:
            count+=1
            
    acc = count/m*100
    return acc

In [311]:
accuracy(NN_predicted,Y_test)

6.811271031826474